In [1]:
import mdtraj as md
import MDAnalysis as mda
import numpy as np
from scipy.stats import ttest_ind_from_stats

from tqdm import tqdm
from pathlib import Path
import os
from natsort import natsorted
import pandas as pd
from addict import Dict as Adict

import pyemma as pm
import deeptime as dt
import deeptime.markov.msm as msm
import deeptime.markov.hmm as hmm

from deeptime.plots import plot_implied_timescales, plot_energy2d, plot_contour2d_from_xyz
from deeptime.markov.sample import *
from deeptime.markov import TransitionCountEstimator
from deeptime.util import energy2d

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import seaborn as sns

from funcs_build_msm import _tica, _kmeans, get_data 
from funcs_validate_msm import *
from funcs_sample import *
from funcs_plotting import *
from funcs_characterise import *
from paths import *

/home/rzhu/Loc/miniconda3/envs/kinase/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
study_name = 'n_clusters'
data_path = Path(f'data_egfr/msm/{study_name}')
summary_f = data_path/f'{study_name}.h5'
hps = pd.read_hdf(summary_f, key='hps')
raw = pd.read_hdf(summary_f, key='result_raw')

summary = raw.groupby('hp_id')[['t2', 'gap_2', 't3', 'gap_3']].agg(['mean', 'std'])
summary

t2                   gap_2                        t3  \
               mean           std      mean        std           mean   
hp_id                                                                   
0      4.258766e+06  4.993950e+06  9.877259  10.570731  438539.507225   
1      2.927477e+06  3.859451e+06  7.083355   7.666884  420395.265777   
2      3.770089e+06  5.647987e+06  8.087046  10.971600  476668.578812   
3      3.775350e+06  5.297374e+06  7.257305   8.182101  513894.618174   

                         gap_3            
                 std      mean       std  
hp_id                                     
0      340220.880011  2.368782  1.164973  
1      355997.202171  2.458528  1.548529  
2      398473.507520  2.461085  1.242254  
3      397912.479254  2.509716  1.214368

In [3]:
hp_id = 0
hp_dict = Adict(hps[hps.hp_id == hp_id].to_dict('records')[0])
features = ['dbdist', 'dbdihed', 'aloop', 'ploopdihed', 'achelix']

savedir = Path(f'data_egfr/msm/validation/{study_name}_hp{hp_id}_validate')
savedir.mkdir(exist_ok=True, parents=True)

hp_dict

{'hp_id': 0,
 'trajlen__cutoff': 1000,
 'n__boot': 20,
 'seed': 49587,
 'tica__lag': 10,
 'tica__stride': 1000,
 'tica__dim': 20,
 'cluster__k': 1000,
 'cluster__stride': 1000,
 'cluster__maxiter': 1000,
 'markov__lag': 100}

In [4]:
kmeans_centres = np.load(savedir/f'kmeans_centers.npy', allow_pickle=True)
print(kmeans_centres.shape)
tmat = np.load(savedir/f'msm_tmat.npy', allow_pickle=True)
tmat.shape

(1000, 20)


(1000, 1000)

In [5]:
stat_dist = dt.markov.tools.analysis.stationary_distribution(tmat)
msm_mod = msm.MarkovStateModel(transition_matrix=tmat, stationary_distribution=stat_dist, lagtime=hp_dict.markov__lag)

---

In [12]:
samples = np.load(savedir/'1_to_5_simulation.npy', allow_pickle=True)
save_samples(samples[::10], traj_files, savedir/'transition_1_to_5_simulation.dcd', reference=None)

In [38]:
samples = np.load(savedir/'6_to_5_simulation.npy', allow_pickle=True)
save_samples(samples[::], traj_files, savedir/'transition_6_to_5_simulation.dcd', reference=None)

In [36]:
samples = np.load(savedir/'6_to_5_simulation.npy', allow_pickle=True)
samples.shape

(546, 2)

In [37]:
samples = np.load(savedir/'1_to_5_simulation.npy', allow_pickle=True)
samples.shape

(27394, 2)

In [16]:
samples = np.load(savedir/'1_to_5_simulation.npy', allow_pickle=True)